# Svami Gap Analysis Demo

Call the /gap-analysis endpoint exposed by the Svami orchestrator to surface BigQuery-backed gap metrics directly inside a Kaggle notebook.


## Prerequisites
- Svami orchestrator service is running and reachable from this notebook.
- A valid API token (or disable auth for local testing).
- The 
equests library is available (pre-installed on Kaggle).


In [ ]:
import os
import requests

SVAMI_URL = os.getenv('SVAMI_URL', 'http://localhost:8003')
API_TOKEN = os.getenv('SVAMI_API_TOKEN', 'demo-token')

session = requests.Session()
session.headers.update({'Authorization': f'Bearer {API_TOKEN}'})

print(f'Using Svami at {SVAMI_URL}')


In [ ]:
payload = {
    'topic': 'FastAPI authentication',
    'artifact_type': 'fastapi',
    'limit': 5
}

response = session.post(f"{SVAMI_URL}/gap-analysis", json=payload, timeout=30)
response.raise_for_status()
analysis = response.json()

print('Summary:')
print(analysis['summary'])


In [ ]:
findings = analysis.get('findings', [])

if not findings:
    print('No gaps returned for this topic.')
else:
    print(f"
Top {len(findings)} findings:")
    for idx, finding in enumerate(findings, start=1):
        location = finding.get('source_url') or finding.get('source_path') or finding.get('chunk_id')
        print(f"
{idx}. {finding['rule_name']} (severity {finding['severity']}, confidence {finding['confidence']:.2f})")
        print(f"   Location: {location}")
        print(f"   Summary : {finding['summary']}")
        if finding.get('suggested_fix'):
            print(f"   Fix     : {finding['suggested_fix']}")


## Interactive gap analysis dashboard
Use the widget below to query Svami's `/gap-analysis` endpoint and preview the bundled heat maps and progress charts.  
Set the `GAP_ANALYSIS_OFFLINE=1` environment variable inside Kaggle to avoid external calls.

In [ ]:
from examples.gap_analysis_widgets import create_gap_analysis_dashboard

create_gap_analysis_dashboard()